# Introduction
This documentation will go through building up a new testing framework from the
ground up. This will start with first describing the current framework and then
move on to rewriting the framework.

## Problem Statement
To reimagine Requirement Verification Tests (RVTs) moving from
an imperative style to a declarative one. What do we mean by imperative vs declarative?
In their current form RVTs describe in extreme detail how to execute a test. 
They describe how to setup and how to configure the simulation, how to setup any 
preconditions, and when to restart a simulation. With all this detail it is usually
difficult to clearly see which lines are the true test and which lines are purely
setup. Additionally because starting a simulation is usually expensive tests are
organized to reuse a simulation which couples tests together in unspecified ways. This
means that when a test fails it can be hard to trace why. 

A declarative approach on the other hand would specify the conditions necessary for the
test to be valid and let the testing apparatus to either figure out how to 
setup those conditions or to recognize when those conditions are valid and execute 
the test. 

This should provide atleast 3 immediate benefits:

1. Quicker test execution both in development and batches as the simulation in 
   most cases will not have to be restarted.
2. Tests that are not coupled
3. Clearer tests. 



## RVT Today
This section will describe what RVT is today. It will build from the ground up
the current iteration of the automation-v2 framework for us to use as a base 
going forward.

An RVT test is simply a set of single line expressions. Each line resolves to 
some code that performs some action. Here is an example RVT in the original
rvt plain text format:


In [86]:
rvt_plaintext = r'''
# Simple Test
StartSimulation

# Precondition
Verify Target.Power False
SetValue Target.Power False


# Verify power is commanded on
# $reqid-123
SendCmd PowerOn
Verify Target.Power \
True

StopSimulation
'''

There is also a yaml format that is supported. It provides a little more 
struture to a test but in the end it simply gets converted into a
plaintext rvt test. One thing to note is that the yaml format started to
introduce a concept of preconditions. This concept was never really flushed
out. The predconditions also just get converted straight to plaintext format
with not special logic.

In [82]:
rvt_yaml = '''
automation:
- test: Setup
  steps:
  - blocks:
    - StartSimulation
    
- test: 
  description: Simple Test
  preconditions:
  - predicates:
    - Verify Target.Power False
    actions:
    - SetValue Target.Power False
  steps:
  - description: |
       Verify power is on
       $reqid-123
    blocks:
    - SendCmd PowerOn
    - Verify Target.Power True
  - blocks:
    - StopSimulation
'''

Since we have two formats we will seperate the reading of the formats
from the evaluation of the test.

## Plaintext Reader
To read a plaintext format we simply can use the exising `shlex` module to
do most of the work. The only other noteable feature is that we support line
continuation. To continue a line simply end the like with `\` character.

In [90]:
#export
import shlex

def read_plaintext(source):
    
    # handle line continuation
    source = source.replace('\\\n', '')

    return [shlex.split(line) 
            for line in source.splitlines()]


We now get an array of lines that are split by the rules of shell lexer. 

In [91]:
read_plaintext(rvt_plaintext)

[[],
 ['#', 'Simple', 'Test'],
 ['StartSimulation'],
 [],
 ['#', 'Precondition'],
 ['Verify', 'Target.Power', 'False'],
 ['SetValue', 'Target.Power', 'False'],
 [],
 [],
 ['#', 'Verify', 'power', 'is', 'commanded', 'on'],
 ['#', '$reqid-123'],
 ['SendCmd', 'PowerOn'],
 ['Verify', 'Target.Power', 'True'],
 [],
 ['StopSimulation']]

## Yaml Reader
Reading yaml is not much harder. We simply will iterate through each test, 
precondition, step and block and output a list of these that we have run through `shlex.split`.

In [65]:
#export
import yaml

def read_yaml(source):
    blocks = []
    
    def description(test_or_step):
        desc = test_or_step.get('description', '')
        if desc:
            return ['# {}'.format(line) for line in desc.splitlines()]
        return []
    
    tests = yaml.safe_load(source).get('automation', [])
    for test in tests:
        blocks += description(test)
        for precondition in test.get('preconditions', []):
            blocks += precondition.get('predicates', [])
            blocks += precondition.get('actions', [])
        for step in test.get('steps', []):
            blocks += description(step)
            blocks += step.get('blocks', [])
        
    return [shlex.split(block) for block in blocks]
        

In [64]:
read_yaml(rvt_yaml)

[['StartSimulation'],
 ['#', 'Simple', 'Test'],
 ['Verify', 'Target.Power', 'False'],
 ['SetValue', 'Target.Power', 'False'],
 ['#', 'Verify', 'power', 'is', 'on'],
 ['#', '$reqid-123'],
 ['SendCmd', 'PowerOn'],
 ['Verify', 'Target.Power', 'True'],
 ['StopSimulation']]

This output closely matches the plaintext output except the plaintext version has a few more empty
lines. 

At this point we might might want to define a few helper methods that can filter out 
empty lines and comments

In [126]:
#export

def is_empty(expr):
    return len(expr) == 0

def is_comment(expr):
    return len(expr) and expr[0].strip().startswith('#')

In [127]:
assert is_empty([])
assert not is_empty(['ABC', '123'])

assert is_comment(['#', 'ABC'])
assert is_comment(['#ABC', '123'])
assert not is_comment(['ABC', '123'])

## Building Blocks
We can now build a function to execute a test. But what exactly are we executing?
Currently our readers are not very sophisticated. Each expression is just a list
of strings. This really won't let us call general python functions that expect things
other than strings. Additionally we want use the arguments as a way to dispatch to
the proper function.

We are going to introduce the `BuildingBlock` class. Each expression in a RVT test 
will resolve to a single building block. 


In [137]:
#export

class BlockResult(object):
    def __init__(self, passed, stdout="", stderr=""):
        self.passed = passed
        self.stdout = stdout
        self.stderr = stderr
        
    def __bool__(self):
        return self.passed
    
    def __str__(self):
        return "<BlockResult: %s, %s, %s>" % ('PASS' if self.passed else 'FAIL', self.stdout, self.stderr)
    
    
class BuildingBlock(object):
    def name(self):
        '''Returns the name of the building block. The name is used
        as a first order lookup for the block'''
        return type(self).__name__
    
    def check_syntax(self, *args):
        '''Returns True if this BuildBlock can support the arguments and False otherwise'''
        return True
    
    def execute(self, *args):
        '''Executes the block. Returns a BlockResult'''
        return BlockResult(False)
    
 

We need to define a few BuildingBlocks to test with.


In [138]:
class EchoBlock(BuildingBlock):
    def execute(self, *args):
        stdout = "%s, args: %r" % (self.name(), [str(arg) for arg in args])
        return BlockResult(True, stdout)
        
class StartSimulation(EchoBlock): pass
class StopSimulation(EchoBlock):  pass
class Verify(EchoBlock): pass
class SetValue(EchoBlock): pass
class SendCmd(EchoBlock): pass


Additionally we need a way to find a building block given an expression.

In [139]:
#export

def all_subclasses(cls):
    return set(cls.__subclasses__()).union(
        [s for c in cls.__subclasses__() for s in all_subclasses(c)])

def find_buildingblock(expr):
    name = expr[0]
    args = expr[1:]
    blocks = [b() for b in all_subclasses(BuildingBlock)]
    matching = [b for b in blocks if b.name() == name and b.check_syntax(*args)]
    if matching:
        return matching[0]

In [140]:
assert StartSimulation == type(find_buildingblock(['StartSimulation']))
assert Verify == type(find_buildingblock(['Verify', '1', '2']))
assert find_buildingblock(['Unknown']) is None

Now we should be able to find a 

Now we can write the executor

In [141]:
#export

def execute_test(expressions):
    for expr in expressions:
        if is_empty(expr) or is_comment(expr):
            continue
            
        block = find_buildingblock(expr)
        if block:
            result = block.execute(expr[1:])
        else:
            result = BlockResult(False, stdout="Could not find block")
          
        # End test
        if not result:
            return result
            

Lets try and execute a test from before

In [144]:

assert execute_test(read_plaintext(rvt_plaintext)) is None
assert execute_test(read_yaml(rvt_yaml)) is None

failing_rvt = '''
Verify Something
Unknown 123
'''

assert execute_test(read_plaintext(failing_rvt)).passed == False

# Observers
Executing a test simply results in None if all tests have passed or a failed block result. 
We will introduce the concept of observers